In [1]:
import pandas as pd
import nannyml as nml
import numpy as np


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# print the path of nml
print(nml.__path__)
print(nml.__version__)

['/Users/carterblair/NannyMLdev/nannyml/nannyml']
0.8.2


In [4]:
def _estimate_chunk(chunk: int):
    # make an empty dictionary
    dict = {}
    for i in range(4):
        # make a dictionary with the chunk number and the value from multiplying it bny i
        dict[f"{i}"] = chunk * i
    print(dict)
    print()
    return dict


In [5]:
chunks = [1, 2, 3]

res = pd.DataFrame.from_records(
    [
        {
            'key': 'test',
            **_estimate_chunk(chunk),
        }
        for chunk in chunks
    ]
)


{'0': 0, '1': 1, '2': 2, '3': 3}

{'0': 0, '1': 2, '2': 4, '3': 6}

{'0': 0, '1': 3, '2': 6, '3': 9}



In [6]:
res


,key,0,1,2,3
0,test,0,1,2,3
1,test,0,2,4,6
2,test,0,3,6,9


In [7]:
dict1 = {'0': 0, '1': 1, '2': 2, '3': 3}
dict2 = {'5': 5, '6': 6, '7': 7, '8': 8}


In [8]:
dict1.update(dict2)
dict1


{'0': 0, '1': 1, '2': 2, '3': 3, '5': 5, '6': 6, '7': 7, '8': 8}

# ****************************

In [18]:
y_true = [0, 1, 1, 0, 1, 1, 0, 1, 0]
y_pred = [0, 1, 0, 0, 1, 1, 1, 0, 0]

# find number of true positives
print("num tp: ", np.sum(np.logical_and(y_true, y_pred)))
print("num tn: ", np.sum(np.logical_and(np.logical_not(y_true), np.logical_not(y_pred))))
print("num fp: ", np.sum(np.logical_and(np.logical_not(y_true), y_pred)))
print("num fn: ", np.sum(np.logical_and(y_true, np.logical_not(y_pred))))


num tp:  3
num tn:  3
num fp:  1
num fn:  2


# **************************************

In [8]:
ref_df, ana_df, _ = nml.load_synthetic_binary_classification_dataset()


In [9]:
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    problem_type='classification_binary',
    metrics=[
        'roc_auc',
        'f1',
        'precision',
        'recall',
        'specificity',
        'accuracy',
        'true_positive',
        'true_negative',
        'false_positive',
        'false_negative',
    ],
    normalize_confusion_matrix=None,
    chunk_size=20_000,
).fit(ref_df)
result = cbpe.estimate(ana_df)
df = result.to_df()

sut = result.filter(period='analysis').to_df()[[('chunk', 'key')] + [(m.column_name, 'value') for m in result.metrics]]


print("(")
print("\t\t\t{'chunk_size\':20000, \'cm_normalization\' = None},")
print("\t\t\tpd.DataFrame(")
print("\t\t\t\t{")
# print the chunk keys in this format: 'key': ['[0:19999]', '[20000:49999]']
print("\t\t\t\t\t\'key\'", ":", sut['chunk']['key'].unique().tolist(), ",")

# print the column names and the list of values in this format: 'roc_auc': [0.5, 0.5] using f strings
for col in sut.columns:
    if col[0] != 'chunk':
        print(f'\t\t\t\t\t\'estimated_{col[0]}\': {sut[col].values.tolist()},')

print("\t\t\t\t}")
print("\t\t\t),")
print("\t\t),")


TypeError: BinaryClassificationAUROC.__init__() got an unexpected keyword argument 'normalize_confusion_matrix'